# The structure of the notebook

There are four sections of the notebook:

* Services: command line, modules, logging
* Global variables
* Calculations
* Functions and tools

## Notes for Tobias:

The order of action:

* Run "Services" (section 2)
* Run the "functions and tools" section (section 5)
* Input the global variables - the couplings, the size, etc (section 3.1)
* Run theta calculator (section 4.1)

(run all the cells inside)

# Services

## Command line

In [17]:
!

## External modules

In [304]:
import itertools
import numpy as np
import math
import random
import matplotlib.pyplot as plt
import time
import re
from scipy import optimize
from functools import reduce
from operator import *

## Logging and debugging

In [224]:
debugging_mode=True

In [225]:
def log(phrase, display_outside_debugging_mode=False):
    '''
    phrase - fstring describing the message
    display_outside_debugging_mode - boolean, explaining whether this line should be printed outside debugging mode
    '''
    if ( (debugging_mode==True) | (display_outside_debugging_mode) ):
        print(phrase)
        print()

# Global variables

## Setting global variables from Tobias-style input

In [336]:
J=0.2

#PT orders that you want to approximate with your initialization

max_PT_order=4

spin_amount=4

input_couplings=[['X1X2',0.2],['X2X3',0.2],['X3X4',0.2]]

# input_generators=['Y1X2','Y2X3','Y3X4','Y1X3','Y2X4', 'Y1X4' , 'Y1X2X3X4']
input_generators=['Y1X2','Y2X3','Y3X4','Y1X3','Y2X4']

### Boring calculations and conversions

In [337]:
computational_states=[list(computational_state) for computational_state in (itertools.product(*[[0,1] for spin in range(spin_amount)]))]

coupling_string_list=[]

coupling_strength_list=[]

for coupling in input_couplings:

    Tobias_coupling_string=coupling[0]
    
    coupling_strength=coupling[1]
    
    assert(coupling_strength==J)

    processed_coupling_string=[[Tobias_coupling_string[2*symbol_id], eval(Tobias_coupling_string[2*symbol_id+1])]
          for symbol_id in range(len(Tobias_coupling_string)//2)]

    Yaroslav_coupling_string=['I' for spin in range(spin_amount)]

    for sub in processed_coupling_string:

        Yaroslav_coupling_string[sub[1]-1]=sub[0]
    
    coupling_string_list+=[Yaroslav_coupling_string]
    
    coupling_strength_list+=[coupling_strength]
    

generator_string_list=[]

for Tobias_generator_string in input_generators:


    processed_generator_string=[[Tobias_generator_string[2*symbol_id], eval(Tobias_generator_string[2*symbol_id+1])]
          for symbol_id in range(len(Tobias_generator_string)//2)]

    Yaroslav_generator_string=['I' for spin in range(spin_amount)]

    for sub in processed_generator_string:

        Yaroslav_generator_string[sub[1]-1]=sub[0]
    
    generator_string_list+=[Yaroslav_generator_string]
    



generator_to_theta_dictionary=[[i] for i in range(len(input_generators))]

In [338]:
unitary_generators=generator_string_list

couplings=coupling_string_list

couplings_amount=len(couplings)

number_of_thetas=len(unitary_generators)

generator_to_theta_dictionary=[[i] for i in range(len(unitary_generators))]

assert tuple(sorted([theta for generator_thetas in generator_to_theta_dictionary for theta in generator_thetas])
            ) == tuple(range(number_of_thetas))

assert len(generator_to_theta_dictionary)==number_of_thetas

### Checking that the conversion went well:

In [293]:
print(generator_string_list, '\n', coupling_string_list)

[['Y', 'X', 'I', 'I'], ['I', 'Y', 'X', 'I'], ['I', 'I', 'Y', 'X'], ['Y', 'I', 'X', 'I'], ['I', 'Y', 'I', 'X']] 
 [['X', 'X', 'I', 'I'], ['I', 'X', 'X', 'I'], ['I', 'I', 'X', 'X']]


# Calculations

## Obtaining theta

### Setting up theta-equations

4 spins
3 first order diagrams

In [362]:
def equation_initialize (PT_mode):
    
    C_series=normalize_C_dictionary(unnormalized_C_dictionary())

    eq_adapted_C_series=eq_adapt_C_series(C_series, PT_mode)

    theta_k_variable_list, f_theta_set_for_eq = f_theta_set_function(PT_mode)

    f_dict_for_eq = f_theta_to_K_s_dict(f_theta_set_for_eq)

    list_of_equations=list(eq_adapted_C_series)
    
    for key in f_dict_for_eq:
    
        if key in eq_adapted_C_series:

            pass

        else:

            eq_adapted_C_series[key]=0
        
    for key in eq_adapted_C_series:

        if key in f_dict_for_eq:

            pass

        else:

            f_dict_for_eq[key]={}      
    
    return(eq_adapted_C_series, theta_k_variable_list, f_theta_set_for_eq, f_dict_for_eq, list_of_equations)

In [363]:
PT_mode='scalar'

eq_adapted_C_series, theta_k_variable_list, f_theta_set_for_eq, f_dict_for_eq, list_of_equations = equation_initialize (PT_mode)

In [364]:
equation_debugging=False

log('the variables are:', equation_debugging)

for variable in theta_k_variable_list:
    
    log(f'{variable}', equation_debugging)
    
log('the theta-side of the equation is:', equation_debugging)

for equation in f_dict_for_eq:
    
    log(f'{equation}: {f_dict_for_eq[equation]}', equation_debugging)

log('the C-side of the equation is:', equation_debugging)

for equation in eq_adapted_C_series:
    
    log(f'{equation}: {eq_adapted_C_series[equation]}', equation_debugging)


### Solving the equations for thetas

In [384]:
def equation_system(theta_k_values):
    
#     For some reason ignoring the J**K weight during the optimization is sometimes beneficial from the perspective of the
#     _weighted_ function. I will comment this part out for now, but we may look at it in closer detail later.
    
#     list_of_eq_values=[np.abs(theta_function(theta_k_values, f_dict_for_eq[equation], theta_k_variable_list)
#                  - eq_adapted_C_series[equation])  
#                        for equation in list_of_equations]
    
#   "eval(re.search('K = \[(.+?)\]',equation).group(1))" is the value of K in the equation
    
    list_of_eq_values=[np.abs(theta_function(theta_k_values, f_dict_for_eq[equation], theta_k_variable_list)
                 - eq_adapted_C_series[equation])  * (J**eval(re.search('K = \[(.+?)\]',equation).group(1)) )
                       for equation in list_of_equations]
    
    if len(list_of_equations)<len(theta_k_variable_list):
        list_of_eq_values+=[0 for additional_equation in range(len(theta_k_variable_list)-len(list_of_equations))]
    
    log(f'list_of_eq_values={list_of_eq_values}')
    
    return(np.array(list_of_eq_values, dtype='float'))

stopwatch=time.time()

theta_k_values=optimize.minimize(lambda theta: sum(equation_system(theta)**2), np.array([0 for k_theta in theta_k_variable_list])).x

# theta_k_values=optimize.leastsq(equation_system, np.array([0 for k_theta in theta_k_variable_list]))[0]

print(f'time elapsed: {time.time()-stopwatch}\n', theta_k_values, equation_system(theta_k_values), '\n The equations are solved with precision: \n', sum(equation_system(theta_k_values)**2))

time elapsed: 3.399350881576538
 [ 2.14687118e-05  2.49975360e-01  0.00000000e+00 -1.65080610e-12
  0.00000000e+00  4.39290989e-04  0.00000000e+00  0.00000000e+00
 -1.86513202e-01  2.14654285e-05 -4.70295073e-04  0.00000000e+00
  2.49945536e-01 -1.87457786e-01  2.50023554e-01  0.00000000e+00
 -6.49441277e-05 -1.65080610e-12  0.00000000e+00  0.00000000e+00] [4.71073797e-06 4.92808731e-06 1.08928060e-05 7.90990984e-07
 1.37941181e-06 5.55425201e-07 3.94610453e-05 3.31880504e-04
 4.12896587e-04 8.16413758e-05 4.98094294e-04 2.28249985e-05
 2.77641081e-07 1.62509656e-05 2.29398075e-05 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00] 
 The equations are solved with precision: 
 5.384280289059716e-07


In [366]:
theta_values=[0 for theta_label in range(len(unitary_generators))]

for contribution_number in range(len(theta_k_variable_list)):
    
    contribution_label=theta_k_variable_list[contribution_number]
    
    theta_index=contribution_label[0]
    
    degree_of_PT=contribution_label[1][0]
    
    contribution_value=theta_k_values[contribution_number]

    theta_values[theta_index]+=contribution_value*J**degree_of_PT

### Here's the list of initialization thetas!

In [ ]:
theta_values

## Hamiltonian diagonalization

### Hamiltonian definition

In [74]:
def single_body_terms():
    
    term_strings=[['Z' if spin_label==term_spin_label else 'I' for spin_label in range(spin_amount)] for term_spin_label in range(spin_amount)]
    
    return(sum([-1.*operator_from_pauli_string(term) for term in term_strings]))


def coupling_terms(J):
    
    return(sum([J*operator_from_pauli_string(term) for term in couplings]))

def H(J):
    
    return(single_body_terms()+coupling_terms(J))


def GS(J):

    GS_E=np.linalg.eigh(H(J))[0][0]

    GS_WF=np.linalg.eigh(H(J))[1].T[0]
              
    return(GS_E, GS_WF)

### Plotting

In [75]:
# plt.plot([np.linalg.eigh(H(J)/(1+np.abs(J)))[0][0:4] for J in np.linspace(-10,10,20)])

In [76]:
# J=-0.5

# plt.plot([np.log(abs(wavefunction_from_PT_series (normalized_C_dictionaries[PT_order], J) @ H(J) @ wavefunction_from_PT_series (normalized_C_dictionaries[PT_order], J) - GS(J)[0])/abs(GS(J)[0]))  for PT_order in range(max_PT_order)])

# plt.plot([np.log(abs(1-abs(wavefunction_from_PT_series (normalized_C_dictionaries[PT_order], J)@GS(J)[1] ))) for PT_order in range(max_PT_order)])

In [77]:
# plt.plot([np.log(abs(wavefunction_from_PT_series (normalized_C_dictionaries[max_PT_order], J) @ wavefunction_from_PT_series (normalized_C_dictionaries[max_PT_order], J)))  for max_PT_order in range(4)])

# Tools and functions

## General tools

### Combinatorics 

In [21]:
def partitioning (n):
    '''
    Creates all possible partitions of an integer n, as a list of lists
    '''
    partitions=[[n]]
    

    
    while partitions[-1][0]>1:
    
        partition=partitions[-1]

        '''
        Finding the rightmost non-one, reducing it by one:
        '''
            
        for k in range(len(partition)):

            if partition[k]>1:
                hit_k=k

            else:
                break
        
        hit_p=partition[hit_k]-1

        rest=sum(partition[hit_k+1:])+1
        
        '''
        Stacking up the rest:
        '''
        
        assemble=[hit_p for i in range(rest//hit_p) ]

        if rest-sum(assemble)!=0:
            assemble+=[rest-sum(assemble)]
        
        '''
        Gathering up the new partition:
        '''
        
        partitions+=[partition[:hit_k]+[hit_p]+assemble]
        
    

    return(partitions)

def fill_the_list(the_list, full_length, filler=0):
    
    if len(the_list)<=full_length:
        return(the_list+[0 for iterator in range(full_length-len(the_list))])
    else:
        return(None)

### k vectors, couplings and generators

In [165]:
def list_of_Ks_from_PT_order(PT_mode='vector'):
    
    
    if PT_mode=='vector':
        
        list_of_Ks=[]

        for mod_K in range(1,max_PT_order+1):
            log(f'list_of_Ks addition={list(partitioning(mod_K))}')
            list_of_Ks+=list(partitioning(mod_K))

        list_of_Ks=[fill_the_list(K, couplings_amount) for K in list_of_Ks if fill_the_list(K,couplings_amount)!=None]

        '''
        including all permutations of the Ks:
        '''

        list_of_Ks=[list(K) for K in list(set(reduce(lambda x, y: x+y, [list(itertools.permutations(K_lexic)) for K_lexic in list_of_Ks])))]

        list_of_Ks=sorted(list_of_Ks, key = lambda K: sum(K))
        
    elif PT_mode=='scalar':
        
        list_of_Ks=[[PT_order] for PT_order in range(1,max_PT_order+1)]
    
    else:
        
        raise Exception('Unknown PT_mode!')
        
    return(list_of_Ks)

def k_to_couplings(k):
    
    '''
    Assumes that the couplings are commuting Paulis, and implicitly that only one coupling i has k_i%2==1
    
    Returns a list of coupling labels as integers from 1 to N_c
    '''
    
    k=list(k)
    
    return([coupling_label for coupling_label in range(len(k)) if k[coupling_label]%2==1])

def pauli_strings_from_couplings_list(coupling_labels):
    
    '''
    Returns a list of pauli strings corresponding to a list of coupling labels
    
    uses global variable 'couplings'
    '''
    
    log(f'pauli_strings_from_couplings_list: \n The coupling labels are: \n {coupling_labels}; \n The couplings are: \n {couplings}')
    
    return([couplings[coupling_label] for coupling_label in coupling_labels])


def k_to_generator(k):
    
    '''
    Assuming odd-even logic in the couplings. 
    k is either list or a numpy array
    '''
    
    k=list(k)
    
    for element in k:
        if element%2==1:
            return k.index(element)
        
    print('Didn"t find a generator for k!')
    
    return(None)

def theta_to_generator(theta_label):
    
    for thetas_per_generator in generator_to_theta_dictionary:
    
        if theta_label in thetas_per_generator:
        
            return(generator_to_theta_dictionary.index(thetas_per_generator))
    
    raise Exception('Wrong input: nonexistent theta!')

def pauli_strings_from_generator_list(generator_list):
    return([unitary_generators[generator_label] for generator_label in generator_list])

### Pauli action

In [23]:
def operator_from_pauli_string(string):
    
    single_qubit_operator_list=[]
    
    for element in string:
        if element=='X':
            single_qubit_operator_list+=[np.array([[0,1],[1,0]])]
        elif element=='Y':
            single_qubit_operator_list+=[np.array([[0,-1j],[0,1j]])]
        elif element=='Z':
            single_qubit_operator_list+=[np.array([[1,0],[0,-1]])]
        elif element=='I':
            single_qubit_operator_list+=[np.array([[1,0],[0, 1]])] 
        else:
            raise('Non-pauli input')
    
    return(reduce(np.kron, single_qubit_operator_list))

def single_pauli_action(pauli, spin):
    
    if pauli=='X':
        return(np.mod(spin+1,2), 1)
    elif pauli=='Y':
        return(np.mod(spin+1,2), 1j*(-1)**spin)
    elif pauli=='Z':
        return(spin, (-1)**spin)
    elif pauli=='I':
        return(spin, 1)
    else:
        print('wrong pauli!')
        return(None)

def pauli_string_action(pauli_string, spins_and_prefactor):
    
    '''
    Given a pauli_string (label) and a computation basis state+prefactor, 
    returns a new computational state with a new prefactor
    
    spins_and_prefactors=[integer list, complex number]
    '''
    
    
    spins=spins_and_prefactor[0]
    
    assert len(pauli_string)==len(spins)
    
    new_spins_and_prefactor=[single_pauli_action(pauli_string[spin_number], spins[spin_number]) for spin_number in range(len(spins))]
    
    return([[element[0] for element in new_spins_and_prefactor], 
                         spins_and_prefactor[1]*reduce(lambda x,y: x*y, [element[1] for element in new_spins_and_prefactor])])


def threaded_pauli_strings_action(pauli_strings,spins_and_prefactor):
    return(reduce(lambda s_and_p, p_str: pauli_string_action(p_str, s_and_p), [spins_and_prefactor]+pauli_strings) )

## PT series

### Dyson elementaries

In [167]:
def s_of_k(K):
    
    log(f's_of_k:\n K={K},\n k_to_couplings(K) = {k_to_couplings(K)}')
    
    return(threaded_pauli_strings_action(pauli_strings_from_couplings_list(k_to_couplings(K)), [[0 for spin in range(spin_amount)],1])[0])

def E0_of_s(s):
    return( sum([-1*(-1)**spin_value for spin_value in s]))

def K_trivial_state_check(K):
    return( reduce(lambda A, B: A and B, [s==0 for s in s_of_k(K)] ) and not reduce(lambda A, B: A and B, [Ki==0 for Ki in K] ))

### C-series manipulations

In [37]:
def C_mult(C1, C2):
    return([C1[0]+C2[0], C1[1]*C2[1]])

def PT_cutoff(C_series):
    
    return([C for C in C_series if np.sum(C[0])<=max_PT_order])

def C_series_add(C_series_1, C_series_2):
    
    the_sum=C_series_1+C_series_2
    
    K_set={tuple(C[0]) for C in the_sum}

    
#     log(f'add={add}')
#     log(f'K_set={K_set}')
    
    the_sum=[[np.array(K), sum([C[1] for C in the_sum if tuple(C[0])==K])] for K in K_set]
    
    the_sum=sorted(the_sum, key=lambda C: np.sum(C[0]))
    
    return(the_sum)

def C_series_mult(C_series_1, C_series_2):
    
    the_product=[[C1[0]+C2[0], C1[1]*C2[1]] for C1, C2 in list(itertools.product(C_series_1, C_series_2))]
    
    K_set={tuple(C[0]) for C in the_product}

    
#     log(f'mult={mult}')
#     log(f'K_set={K_set}')
    
    the_product=[[np.array(K), sum([C[1] for C in the_product if tuple(C[0])==K])] for K in K_set]
    
    the_product=sorted(the_product, key=lambda C: np.sum(C[0]))
    
    return(the_product)

### Dyson calculus: unnormalized C-series

In [38]:
def unnormalized_C_dictionary():
    
    list_of_Ks=list_of_Ks_from_PT_order()
    
    the_C_list=[[np.array([0 for coupling in range(couplings_amount)]), 1]]
    the_C_dictionary={str(a_C[0].tolist()): a_C[1] for a_C in the_C_list}

    delta_betas=[np.array(delta_beta) for delta_beta in np.eye(couplings_amount, dtype='int').tolist()]

    for K_as_a_list in [K for K in list_of_Ks if ((not K_trivial_state_check(K)) and (sum(K)<=max_PT_order))]:
                
        
        K=np.array(K_as_a_list)

        log(f'K={K}')

        K_betas=[K-delta_betas[beta] for beta in range(couplings_amount) 
                 if K[beta]>0 and not K_trivial_state_check(K-delta_betas[beta])]

        C_betas=[the_C_dictionary[str(K_beta.tolist())] for K_beta in K_betas]

        log(f'K_betas={K_betas}')

        k_primes=[np.array(k_prime) for k_prime in list(itertools.product(*[list(range(Ki+1)) for Ki in K]))]

        k_primes=[k_prime for k_prime in k_primes if (K_trivial_state_check(k_prime))]

        k_primes_betas=[[k_prime-delta_betas[beta] for beta in range(couplings_amount) if k_prime[beta]>0] for k_prime in k_primes]


        log(f'k_primes={k_primes}')
        log(f'k_primes_betas={k_primes_betas}')

        C_k_prime_beta_sums=[sum([the_C_dictionary[str(k_prime_beta.tolist())] if str(k_prime_beta.tolist()) 
                                  in the_C_dictionary else 0 for k_prime_beta in k_primes_betas[k_prime_index] ]) 
                             for k_prime_index in range(len(k_primes))]
        
        C_k_minus_k_primes=[the_C_dictionary[str((K-k_prime).tolist())] if str((K-k_prime).tolist()) 
                            in the_C_dictionary else 0 for k_prime in k_primes]
      

        k_prime_terms=[C_k_prime_beta_sums[k_prime_iterator]*C_k_minus_k_primes[k_prime_iterator] 
                       for k_prime_iterator in range(len(k_primes))]



        log(f'k_primes_terms={k_prime_terms}')

        the_C_list+=[[K, (sum(C_betas)-sum(k_prime_terms))/(E0_of_s([0 for spin in range(spin_amount)])
                                                            
                                                            -E0_of_s(s_of_k(K)))]]

        log(f'the_C_list addition = {[K, (sum(C_betas)-sum(k_prime_terms))/(E0_of_s([0,0,0,0])-E0_of_s(s_of_k(K)))]}')



        the_C_dictionary={str(a_C[0].tolist()): a_C[1] for a_C in the_C_list}

        log(f'current the_C_dictionary = {the_C_dictionary}')


    return(the_C_dictionary)

### Normalizing, equation-adapting C-series

In [27]:
def C_series_to_Z(C_series):
    
    C_series_by_comp_state={str(s): [C for C in C_series if s_of_k(C[0])==s] for s in computational_states}
    
    log(f'C_series_by_comp_state\n')
    for comp_state in C_series_by_comp_state:
        log(f'{comp_state}={C_series_by_comp_state[comp_state]}')
    
    pre_Z=[C_series_mult(C_series_by_comp_state[str(s)], C_series_by_comp_state[str(s)]) for s in computational_states]
    
    for pre_Z_element in pre_Z:
        log(f'pre_Z_element={pre_Z_element}\n')
    
    Z=reduce(C_series_add, pre_Z)
    
    log(f'Z={Z}')
    
    return(Z)


def N_from_Z(Z):
    
    X=[C_Z for C_Z in Z if np.sum(C_Z[0])!=0]
    
    alpha=-1/2
    
    N=[[np.array([0 for coupling in range(couplings_amount)]), 1]]
    
    expansion_term=[[np.array([0 for coupling in range(couplings_amount)]), 1]]
    
    for order in range(1, max_PT_order+1):
        
        expansion_term=[[C[0], C[1]*(alpha-order+1)/order] for C in PT_cutoff(C_series_mult(expansion_term,X))]
        N=C_series_add(N,expansion_term)
                    
        log(f'added to N: {expansion_term}\n')
        log(f'new N: {N}\n')
    
    N=sorted(N, key= lambda C: np.sum(C[0]))
        
    return( PT_cutoff(N) )


def normalize_C_dictionary(the_C_dictionary):

    the_C_list=[[np.array(eval(key)), the_C_dictionary[key]] for key in the_C_dictionary]

    Z=PT_cutoff(C_series_to_Z(the_C_list))

    N=N_from_Z(Z)

    normalized_C_list=PT_cutoff(C_series_mult(the_C_list, N))

    normalized_C_dictionary={str(a_C[0].tolist()): a_C[1] for a_C in normalized_C_list}



    return(normalized_C_dictionary)



def eq_adapt_C_series(C_series, PT_mode='vector'):
    
    '''
    C_series is assumed to have raw form ({'[0,0,0]': 1, ...}), 
    but the output is in the equation-ready form ({'K = {[K]}, s = {s}': ...})
    '''
    
    adapted_C_series = dict()

    for K in C_series:
        
        if K == str([0 for coupling in range(couplings_amount)]):
            
            continue
        
        if PT_mode=='scalar':
        
            C_K_label=f'K = {[sum(eval(K))]}, s = {s_of_k(eval(K))}'
            
        elif PT_mode=='vector':
            
            C_K_label=f'K = {K}, s = {s_of_k(eval(K))}'
            
        else:
            
            raise Exception('Unknown PT_mode!')

        if C_K_label in adapted_C_series:

            adapted_C_series[C_K_label] += C_series[K]

        else:

            adapted_C_series.update({C_K_label: C_series[K]})

        log(f'included the value of {C_K_label} type into the series: {C_series[K]}')

    return (adapted_C_series)

### Wavefunction representation

In [28]:
def wavefunction_from_PT_series (normalized_C_dictionary, J):
    
    computational_states=[ [[np.array([1 if s==label else 0 for label in range(2)]) for s in s_string], coef] for s_string, coef in [[s_of_k(eval(K)), normalized_C_dictionary[K]*J**sum(eval(K))] for K in normalized_C_dictionary]]
    
    wavefunction=sum([coef*reduce(np.kron, computational_state) for computational_state, coef in computational_states])
    
    wavefunction=wavefunction/np.linalg.norm(wavefunction)
    
    return(wavefunction)

### Testing

In [29]:
# C_coefficients={'[0, 2, 0]': -1/32, '[0, 0, 2]': -1/32, '[2, 0, 0]': -1/32, '[1, 1, 0]': 1/8, '[0, 1, 1]': 1/8, '[1, 0, 1]': 1/16, '[0, 0, 1]': -1/4, '[1, 0, 0]': -1/4, '[0, 1, 0]': -1/4}
# C_coefficients

## f-analysis

### $\vec{K}(f)$, $\vec{N}(f)$ and $\Theta(f)$

In [30]:
def PT_order_from_f_theta(f_theta):
    return(list(sum([np.array(k_theta[1]) for k_theta in f_theta])))

def f_theta_to_pauli_strings(f_theta):
    
    return(pauli_strings_from_generator_list([theta_to_generator(k_theta[0]) for k_theta in f_theta]))

def comp_state_from_f_theta(f_theta):
    
    generators_action=threaded_pauli_strings_action(f_theta_to_pauli_strings(f_theta),
                                                    [[0 for spin_number in range(spin_amount)], 1j**len(f_theta)])
    
    return(generators_action)

def multiplicities_factorials(f_theta):
    counts=[]
    for element in f_theta:
        counts+=[f_theta.count(element)]
        f_theta=list(filter(lambda element_prime: element_prime!=element, f_theta))

    return(reduce(mul, [math.factorial(count) for count in counts]) )
    
def product_function(theta_ks, f_theta, theta_k_variable_list):
    
    theta_k_indices=[theta_k_variable_list.index(k_theta) for k_theta in f_theta]
    
    prefactor=comp_state_from_f_theta(f_theta)[1]/multiplicities_factorials(f_theta)
    
    return(reduce(mul, [theta_ks[index] for index in theta_k_indices] )* prefactor)

def theta_function(theta_ks, fs_theta, theta_k_variable_list):

    return(sum(product_function(theta_ks, f, theta_k_variable_list) for f in fs_theta))

### Combinatoric tools

In [31]:
def odd_count(the_list):
    return(len([element for element in the_list if element%2==1]))

def odd_count_equals_one(theta_k):
    return(odd_count(theta_k[1])==1)
    
def theta_to_k_correspondence(theta_k):
    return(k_to_generator(theta_k[1])==theta_to_generator(theta_k[0]))

def f_theta_PT_filter(f_theta):
    return(sum([sum(theta_k[1]) for theta_k in f_theta])<=max_PT_order)


def theta_k_filter(theta_k, PT_mode):
    
    if PT_mode=='vector':
        return(odd_count_equals_one(theta_k) and theta_to_k_correspondence(theta_k))
    
    elif PT_mode=='scalar':
        return(odd_count_equals_one(theta_k))
    
    else:
        raise Exception('Unknown PT_mode!')

### Draft

$\theta^\alpha$ - a parameter of a single generator

\theta^\alpha = \sum_k \theta^\alpha_k J^k

e^ i \theta T


\Psi = e^i\theta T* ... * ... * \ket{0000} 

$C_\vec{K} + C_\vec{K'} + C_\vec{K''}  = # * \theta^\alpha_k * \theta^\alpha'_k' * ... + \theta^\alpha_k * \theta^\alpha'_k' * ... , k+k'+..=K, K=|\vec{K}|=|\vec{K'}|=|\vec{K''}|, s(\vec{K, K', K''}) = s, s(\alpha,\alpha') = T^\alpha T^\alpha'..{0000} $

K=(3,0,0)

K'=(1,0,2)

K''=(0,0,3)

1100

theta^\alpha_(1,0,0)=0

T=IYXI

### f list generation

The scheme of f_theta creation:

* Generate all theta_k, i.e. $\theta^{\vec{k}}$ (or $\theta^{k}$); list as first order terms; remove the apriori bad theta_ks by some appropriate filters
* Take a Descartes square of that; throw away the dublicates and all above the PT filter; list as second power terms
* Multiply 2nd order by the first order terms; remove the dublicates, do the PT filter; list as third power
* Etc, until the power=max PT order. Combine the results together into a full f_theta set
* Output the theta_ks and the full set

The dictionary corresponding the f_thetas and the K vectors they amount to, is created separately, using $K(f)$

In [246]:
def f_theta_set_function(PT_mode):
    
    stopwatch=time.time()
    
    PT_orders_for_theta=list_of_Ks_from_PT_order(PT_mode)
    
    log(f'PT_orders_for_theta: {PT_orders_for_theta}')
    
    theta_k_set=set()

    for theta in range(number_of_thetas):
        theta_k_set.update([(theta, tuple(a_K)) for a_K in PT_orders_for_theta ])
    
    log(f'theta_k_set: {theta_k_set}')
    
#     For TUCC applied to TFIM, it makes sense to remove all even PT orders from theta expansion
#     theta_k_set=set(filter(lambda theta_k: theta_k_filter(theta_k, PT_mode), theta_k_set))

    log(f'filtered theta_k_set: {theta_k_set}')
    
    all_f_thetas=set()

    new_power_f_thetas=set((theta_k,) for theta_k in theta_k_set)

    log(f'new_power_f_thetas: {new_power_f_thetas}')

    all_f_thetas.update(new_power_f_thetas)

    for theta_power in range(1, max_PT_order+1):   

        '''
        Simply listing all potential next-power terms
        '''

        new_power_f_thetas=set(a_product[0]+(a_product[1],) for a_product 
                               in set(itertools.product(new_power_f_thetas,theta_k_set)) )

        log(f'new_power_f_thetas: {new_power_f_thetas}')


        '''
        Sorting the f_thetas: to avoid dublicates and to order terms for the T-action
        '''

        new_power_f_thetas=set(tuple(sorted(list(f_theta), key=lambda theta_k: theta_k[0])) for f_theta in new_power_f_thetas)

        log(f'sorted new_power_f_thetas: {new_power_f_thetas}')

        '''
        Filtering out the higher order terms
        '''

        new_power_f_thetas=set(filter(lambda f_theta: f_theta_PT_filter(f_theta), new_power_f_thetas))

        log(f'filtered new_power_f_thetas: {new_power_f_thetas}')

        all_f_thetas.update(new_power_f_thetas)

        log(f'all_f_thetas: {all_f_thetas}')
    
    log(f'f_theta_set evaluation completed, time elapsed: {time.time()-stopwatch}')
    
    return(list(theta_k_set), all_f_thetas)





 ... Adding metadata to the possible f functions: K, or K and s 
(K can be either vector or scalar, doesn't matter) ...

In [126]:
def f_theta_to_K_dict(the_f_theta_set):
    
    f_theta_K_dict=dict()
    
    for f_theta in the_f_theta_set:
        
        label_for_f_theta = f'K = {str(PT_order_from_f_theta(f_theta))}'
        
        if label_for_f_theta in f_theta_K_dict:
            
            f_theta_K_dict[label_for_f_theta].update({f_theta})
            
        else:
            
            f_theta_K_dict.update({label_for_f_theta : {f_theta}})
       
        
    return(f_theta_K_dict)

def f_theta_to_K_s_dict(the_f_theta_set):
    
    f_theta_K_s_dict=dict()
    
    for f_theta in the_f_theta_set:
        
        K_for_f_theta=PT_order_from_f_theta(f_theta)
        
        s_for_f_theta=tuple(comp_state_from_f_theta(f_theta)[0])
        
        label_for_f_theta=f'K = {str(PT_order_from_f_theta(f_theta))}, s = {comp_state_from_f_theta(f_theta)[0]}'
        
        if label_for_f_theta in f_theta_K_s_dict:
            
            f_theta_K_s_dict[label_for_f_theta].update({f_theta})
            
        else:
            
            f_theta_K_s_dict.update({label_for_f_theta : {f_theta}})
       
    for K_s in f_theta_K_s_dict:
        log(f'{K_s}: {f_theta_K_s_dict[K_s]}')    
    
    return(f_theta_K_s_dict)

### Faster (but more complicated) f list generation

#### Combinatorics

In [34]:
def length_fillter(list_of_lists, max_length):
    
    '''
    Fills the elements of a list of lists with zeros, up to the certain max_length, 
    and returns None if there are longer elements
    '''
    
    new_list_of_lists=[]
    
    for a_list in list_of_lists:
    
        new_list_of_lists+=[fill_the_list(a_list, max_length, 0)]
        if new_list_of_lists[-1]==None:
            return(None)
        
    return(new_list_of_lists)


    
def odd_upon_even_condition(the_list):

    return(reduce((lambda x, y: x and y), [len([element for element in sublist if element%2==1])==1 for sublist in the_list]))
    
def odd_upon_even_filter(lists_of_k):
    return([a_list for a_list in lists_of_k if odd_upon_even_condition(a_list)])

#### Functions

In [35]:
def fs_from_K (the_K, max_PT_order=max_PT_order):
    '''
    Produces all f-functions such that K(f)=K for a given K-vector
    
    Each f is represented by a list of k-vectors (tuples) which are included in the f, 
    with dublicates representing the multiplicities
    
    '''
    
    '''
    The scheme of the algorithm is as follows:
    
    first, one considers independent partitions of Ki (elements of K)
    
    then, these (and zeros) are used as ki for k-vectors; 
    the number of generators are fixed to be the number of odd ki's
    
    all k-vectors are produced, as the possible 1-1-1-..-1 correspondences between the available kis
    '''
    
    '''
    partitioned_Ki= list of partitions of K_i for each Ki in K
    K_partitions= combinations of partitions of each K_i
    '''

    partitioned_Ki=[]

    log(f"the_K={K}")
    
    for element in the_K:
        partitioned_Ki+=[partitioning(element)]

    K_partitions=[list(K_partition) for K_partition in itertools.product(*partitioned_Ki)]

    
    fs_for_K=set()
    

    for K_partition in K_partitions:

        number_of_generators=odd_count([element for sublist in list(K_partition) for element in sublist])

        '''
        filltered_K_partition - filled with zeros up until the total amount of odds; if not enough odds, throws out the thing
        '''
        
        log(f"K_partition={K_partition}")
        
        
        filltered_K_partition=length_fillter(K_partition, number_of_generators)

        log(f"filltered_K_partition={filltered_K_partition}")
        
        if filltered_K_partition==None:
            continue


        '''
        generates all possible ways to break the filltered_K_partition into k_i correspondences
        '''
           
        K_partition_permutations=[ [tuple(filltered_K_partition[0])] ]+[list(set(itertools.permutations(element))) for element in filltered_K_partition[1:]]

        log(f"K_partition_permutations={K_partition_permutations}")
        
        k_combinations=list( itertools.product(*K_partition_permutations) )

        '''
        k_combinations_as_lists - possible combinations of k^alpha_i in an f, given a partition of K
        '''

        k_combinations_transposed=[[tuple(k) for k in list(np.array(ki_combination).T)] for ki_combination in k_combinations]

        log(f"k_combinations={k_combinations} \nk_combinations_transposed={k_combinations_transposed}")
        
        k_combinations_transposed=odd_upon_even_filter(k_combinations_transposed)

        log(f"k_combinations_transposed filtered={k_combinations_transposed}")
        
        for element in k_combinations_transposed:
            element.sort()

        filtered_tuples_of_k=[tuple(list_of_k) for list_of_k in k_combinations_transposed]

        fs_for_K.update(filtered_tuples_of_k)

        '''
        set_of_ks - same as lists_of_k, but a set in terms of tuples, filtered the ones where odd k_i occurs other than twice, 
                    and the whole tuples never repeat
        '''
        
    return(fs_for_K)


def fs_theta_from_fs(fs):
    
    '''
    Produces all f_theta functions, given the f-functions    
    '''
    
    fs_theta=set()
    
    for f in fs_for_K:
        
        '''
        generate all possible combinations of parameters, corresponding to the list of k's
        '''
        
        
        fs_theta_from_f=list(itertools.product(*[
            
            tuple([(theta_label,k) for theta_label in generator_to_theta_dictionary[k_to_generator(k)] ] ) 
            
            for k in f]))
        
        
        log(f'k-theta distribution:{fs_theta_from_f}')
        
        '''
        sort each final combination according to the order of parameters - 
        to avoid ambiguity in the def of f_theta and to make the generator product ("T-action") well-defined
        '''
        
        fs_theta_from_f=[tuple(sorted(f_theta, key=lambda k_theta: k_theta[0])) for f_theta in fs_theta_from_f]
        
        log(f'sorted k-theta distribution:{fs_theta_from_f}')
        
        '''
        there may be some duplicates in f_theta, but they are removed when updating:
        '''
        
        fs_theta.update(fs_theta_from_f)
        
    return(fs_theta)

NameError: name 'max_PT_order' is not defined

#### Generating the fs

In [ ]:
# log(f'list_of_Ks={list_of_Ks}')

# stopwatch=time.time()

# all_fs=dict()

# all_ks_theta=set()

# all_fs_theta=dict()

# for K in list_of_Ks_from_PT_order(max_PT_order):

#     fs_for_K=fs_from_K(K)
    
#     all_fs.update({str(K): fs_for_K})
    
#     fs_theta_for_K=fs_theta_from_fs(fs_for_K)

#     for f_theta in fs_theta_for_K:
                
#         all_ks_theta.update(f_theta)
        
#     all_fs_theta.update({str(K): fs_theta_for_K})


# k_and_theta_list=list(all_ks_theta)            

# for K in all_fs_theta:
#     log(f'{K}: {all_fs_theta[K]}')

# print(time.time()-stopwatch)
